In [35]:
# IMPORTS 
import pandas as pd
import numpy as np
import shutil
import os
import time
import subprocess
import sys
import datetime
import math
import matplotlib.pyplot as plt
from os.path import exists as file_exists
import matplotlib.dates as mdates

In [36]:
# FILE PATHS
file_path = r"C:\Users\govertsen.k\Desktop\trial\typical"
file_path = file_path.replace(os.sep, '/')
file_path

'C:/Users/govertsen.k/Desktop/trial/typical'

In [37]:
# CONSTANTS
study = ["baseline","outage"]

# Identify all the models
models = [os.path.splitext(filename)[0] for filename in os.listdir(file_path + '/baseline/')]

In [38]:
# Outage 
otg_date = "July 24"
otg_hour = "12"
otg_lengths = os.listdir(file_path + "/outage/")
otg_lengths.sort(key=int)
otg_lengths

sample_dates = [['07/24 00:00:00'],['07/26 23:00:00']]
index = ['start','end']
columns = ['Date/Time']
sample = pd.DataFrame(sample_dates,index,columns)
sample['Date/Time']= pd.to_datetime(sample['Date/Time'], format = '%m/%d %H:%M:%S')
   

In [39]:
# Make new summary folders
new_path = file_path + "/" + "summary"
if not os.path.exists(new_path):
    os.makedirs(new_path)

In [40]:
# Save Outdoor Data
outdoor = pd.DataFrame()
data = pd.read_csv(file_path + '/output/baseline/' + models[0] + '.csv')
data = data.loc[data['LIVING ZONE:Zone Air Temperature [C](Hourly)'].notnull()]
outdoor['Date/Time'] = data['Date/Time']
outdoor['Outdoor Temp'] = data['LIVING ZONE:Zone Outdoor Air Drybulb Temperature [C](Hourly)']

# Change '24:00:00' to "00:00:00", set datetime, add one day to midnight values, reset index, and format
outdoor['Date/Time'] = outdoor['Date/Time'].str.strip()
outdoor['Date/Time'] = outdoor['Date/Time'].str.replace('24:00:00','00:00:00')
outdoor['Date/Time'] = pd.to_datetime(outdoor['Date/Time'],format = '%m/%d  %H:%M:%S')
outdoor.loc[outdoor['Date/Time'].dt.hour == 0,'Date/Time'] = outdoor.loc[outdoor['Date/Time'].dt.hour == 0]['Date/Time'] + np.timedelta64(1,'D')
outdoor = outdoor.reset_index(drop=True)

# Trim down just to sample dates
outdoor = outdoor.loc[(outdoor['Date/Time'] >= sample['Date/Time'][0]) & (outdoor['Date/Time'] <= sample['Date/Time'][1])]
outdoor = outdoor.reset_index(drop=True)
    
outdoor.to_csv(file_path + "/summary/outdoor.csv",index=False)

In [41]:
# BASELINES
# - Temperature
baseline_temperature = pd.DataFrame(columns = models)

# - Humidity
baseline_humidity = pd.DataFrame(columns = models)

# - Electricity
baseline_electricity =pd.DataFrame(columns = models)

# - Gas 
baseline_gas = pd.DataFrame(columns = models)


for each_model in models:
    ## Baseline
    # Create empty dataframe
    baseline = pd.DataFrame()

    # Read the baseline output file
    data = pd.read_csv(file_path + '/output/baseline/' + each_model + '.csv')
    data = data.loc[data['LIVING ZONE:Zone Air Temperature [C](Hourly)'].notnull()]
    baseline['Date/Time'] = data['Date/Time']
    baseline['Outdoor Temp'] = data['LIVING ZONE:Zone Outdoor Air Drybulb Temperature [C](Hourly)']
    baseline['RH'] = data['LIVING ZONE:Zone Air Relative Humidity [%](Hourly)']
    baseline['Indoor Temp'] = data['LIVING ZONE:Zone Air Temperature [C](Hourly)']
    baseline['Electricity'] = data['Electricity:Facility [J](Hourly)']
    if 'NaturalGas:Facility [J](TimeStep) ' in data:
        baseline['Gas'] = data['NaturalGas:Facility [J](TimeStep) ']

    # Change '24:00:00' to "00:00:00", set datetime, add one day to midnight values, reset index, and format
    baseline['Date/Time'] = baseline['Date/Time'].str.strip()
    baseline['Date/Time'] = baseline['Date/Time'].str.replace('24:00:00','00:00:00')
    baseline['Date/Time'] = pd.to_datetime(baseline['Date/Time'],format = '%m/%d  %H:%M:%S')
    baseline.loc[baseline['Date/Time'].dt.hour == 0,'Date/Time'] = baseline.loc[baseline['Date/Time'].dt.hour == 0]['Date/Time'] + np.timedelta64(1,'D')
    baseline = baseline.reset_index(drop=True)

    # Trim down just to sample dates
    baseline = baseline.loc[(baseline['Date/Time'] >= sample['Date/Time'][0]) & (baseline['Date/Time'] <= sample['Date/Time'][1])]
    baseline = baseline.reset_index(drop=True)
    
    # Transfer to consilidated list 
    baseline_temperature[each_model] = baseline["Indoor Temp"]
    baseline_humidity[each_model] = baseline["RH"]

    baseline_electricity[each_model] = baseline["Electricity"]
    if 'Gas' in baseline:
        baseline_gas[each_model] = baseline["Gas"]
        
# Save as CSV
baseline_temperature = baseline_temperature.set_index(baseline['Date/Time'])
baseline_humidity = baseline_humidity.set_index(baseline['Date/Time'])
baseline_electricity = baseline_electricity.set_index(baseline['Date/Time'])
baseline_gas = baseline_gas.set_index(baseline['Date/Time'])
baseline_temperature.to_csv(file_path + "/summary/baseline_temperature.csv",index=True)
baseline_humidity.to_csv(file_path + "/summary/baseline_humidity.csv",index=True)
baseline_electricity.to_csv(file_path + "/summary/baseline_electricity.csv",index=True)
baseline_gas.to_csv(file_path + "/summary/baseline_gas.csv",index=True)
baseline = baseline.reset_index(drop=True)


In [28]:
baseline_temperature = baseline_temperature.reset_index(drop=True)
baseline_temperature

,bldg0002642,bldg0003104,bldg0006733,bldg0009153,bldg0010160,bldg0010982,bldg0012954,bldg0013107,bldg0013299,bldg0014965,...,bldg0527700,bldg0528109,bldg0533187,bldg0534041,bldg0535065,bldg0536627,bldg0539572,bldg0539770,bldg0544793,bldg0547838
0,23.888908,25.555558,25.555556,22.218654,23.888891,28.958075,21.107264,33.800505,22.219002,20.581699,...,33.142204,23.888921,23.053432,20.229809,21.111113,25.261523,36.247163,26.090031,18.884183,21.633539
1,23.888907,24.445737,25.398800,22.218881,23.888890,27.882000,21.107640,33.431922,22.005192,20.244148,...,32.611682,23.888916,23.053873,19.993576,21.111113,24.860062,35.942975,25.628845,18.884776,21.063740
2,23.888905,24.444448,25.196662,22.219209,23.868965,26.934493,21.107963,33.026014,21.782486,20.010559,...,31.910785,23.888911,23.054211,19.993498,21.111113,24.530214,35.651229,24.915748,18.885164,20.829233
3,23.888900,24.444447,24.886104,22.219689,23.626525,26.027902,21.108357,32.553746,21.562348,19.995646,...,31.177493,23.888902,23.054519,19.994204,21.111112,24.126983,35.319110,24.709762,18.885494,20.828120
4,23.888896,24.444446,24.617890,22.219997,23.427345,25.263762,21.108608,32.146455,21.336333,19.996095,...,30.497075,23.888895,23.054783,19.994601,21.111112,23.822874,35.026798,24.373349,18.885756,22.390504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,23.888892,25.555557,25.555560,22.230299,26.311468,35.428657,21.565935,37.064850,26.114777,22.440204,...,35.085507,23.888905,23.051678,22.447037,21.111589,24.444658,36.948823,26.666698,18.913055,24.411894
68,23.888894,25.555557,25.555558,22.217154,26.499951,33.928292,21.190701,36.332317,27.048307,22.184029,...,34.274625,23.888907,23.052254,21.803819,21.111310,25.518678,36.633363,26.666689,18.886116,23.499887
69,23.935019,25.637842,25.555560,22.316568,27.358430,33.793429,22.417895,38.676195,28.754075,22.392173,...,33.225643,23.888915,23.052930,20.915915,21.111117,25.421315,35.720360,26.635340,18.886139,22.598202
70,23.890280,25.556880,25.555560,22.222934,25.330183,32.533427,21.823057,38.494906,28.403999,21.735000,...,32.241234,23.888911,23.053726,20.088489,21.111115,24.802960,35.583201,25.977028,18.886460,21.772407


In [34]:
# OUTAGE
for each_otg in otg_lengths:
    # - Temperature
    outage_temperature = pd.DataFrame(columns = models)
    
    # - Humidity
    outage_humidity = pd.DataFrame(columns = models)
    
    # - Temperature Difference
    outage_temp_diff = pd.DataFrame(columns = models)
    
    # - Electricity
    outage_electricity = pd.DataFrame(columns = models)
    
    # - Gas
    outage_gas = pd.DataFrame(columns = models)

    for each_model in models: 

        ## Outage
        # Create empty dataframe
        outage = pd.DataFrame()

        data = pd.read_csv(file_path + '/output/outage/' + each_otg + "/" + each_model + '.csv')
        data = data.loc[data['LIVING ZONE:Zone Air Temperature [C](Hourly)'].notnull()]
        outage['Date/Time'] = data['Date/Time']
        outage['Outdoor Temp'] = data['LIVING ZONE:Zone Outdoor Air Drybulb Temperature [C](Hourly)']
        outage['Indoor Temp'] = data['LIVING ZONE:Zone Air Temperature [C](Hourly)']
        outage['RH'] = data['LIVING ZONE:Zone Air Relative Humidity [%](Hourly)']
        outage['Electricity'] = data['Electricity:Facility [J](Hourly)']
        if 'NaturalGas:Facility [J](TimeStep) ' in data:
            outage['Gas'] = data['NaturalGas:Facility [J](TimeStep) ']

        outage['Date/Time'] = outage['Date/Time'].str.strip()
        outage['Date/Time'] = outage['Date/Time'].str.replace('24:00:00','00:00:00')
        outage['Date/Time'] = pd.to_datetime(outage['Date/Time'],format = '%m/%d  %H:%M:%S')

        outage.loc[outage['Date/Time'].dt.hour == 0,'Date/Time'] = outage.loc[outage['Date/Time'].dt.hour == 0]['Date/Time'] + np.timedelta64(1,'D')

        outage=outage.reset_index(drop=True)

        # Reduce to near outage
        outage = outage.loc[(outage['Date/Time'] >= sample['Date/Time'][0]) & (outage['Date/Time'] <= sample['Date/Time'][1])]
        outage = outage.reset_index(drop=True)
        outage_temperature[each_model] = outage['Indoor Temp']
        outage_humidity[each_model] = outage['RH']
        outage_electricity[each_model] = outage["Electricity"]
        outage_temp_diff[each_model] = outage['Indoor Temp'] - baseline_temperature[each_model]

        if 'Gas' in outage.columns:
            outage_gas[each_model] = outage["Gas"]
        outage = outage.set_index(outage['Date/Time'])

   
    outage_temperature = outage_temperature.set_index(outage['Date/Time'])
    outage_temp_diff = outage_temp_diff.set_index(outage['Date/Time'])
    outage_humidity = outage_humidity.set_index(outage['Date/Time'])
    outage_electricity = outage_electricity.set_index(outage['Date/Time'])
    outage_gas =  outage_gas.set_index(outage['Date/Time'])

    outage_temperature.to_csv(file_path + "/summary/outage_temperature_"+ each_otg + "_hr.csv",index=True)
    outage_temp_diff.to_csv(file_path + "/summary/outage_temp_diff_"+ each_otg + "_hr.csv",index=True)
    outage_humidity.to_csv(file_path + "/summary/outage_humidity_"+ each_otg + "_hr.csv",index=True)
    outage_electricity.to_csv(file_path + "/summary/outage_electricity_" + each_otg + "_hr.csv",index=True)
    outage_gas.to_csv(file_path + "/summary/outage_gas_"+ each_otg +"_hr.csv",index=True)

In [ ]:
# for each_otg in otg_lengths:
#     for each_model in models: 
#          outage_temp_diff[each_model] = baseline_temperature[each_model] - outage_temperature[each_model]
# #     outage_temp_diff =  outage_temp_diff.set_index(outage['Date/Time'])
#     outage_temp_diff.to_csv(file_path + "/summary/outage_temp_diff_" + each_otg + "_hr.csv",index=True)    
